In [ ]:
import os
import re
import nest_asyncio
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Financial_RAG_Chatbot.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/crestereo.py  
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/raft_stereo.py  
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/__init__.py  
   creating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/__pycache__/
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/__pycache__/crestereo.cpython-311.pyc  
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/__pycache__/raft_stereo.cpython-311.pyc  
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision/prototype/models/depth/stereo/__pycache__/__init__.cpython-311.pyc  
  inflating: Financial_RAG_Chatbot/venv/Lib/site-packages/torchvision

In [ ]:
!pip install -r "Financial_RAG_Chatbot/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 1. Install Ollama (Linux/Colab only)
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start the Ollama Server in the background
import subprocess
import time

# Run "ollama serve" as a background process
process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Starting Ollama server...")
time.sleep(10)  # Wait for it to spin up

# 3. Download the Llama3 model (Required)
!ollama pull llama3.1

>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tgz
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama server...



In [ ]:
import os
import operator
from typing import TypedDict, Annotated, List, Union
from dotenv import load_dotenv
import re

# LangChain / LangGraph Imports
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END

# Vector Store Imports (to connect to your data)
from llama_index.core import StorageContext, load_index_from_storage, VectorStoreIndex, Document, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import MarkdownElementNodeParser, MarkdownNodeParser
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb

#Parser
from llama_parse import LlamaParse

#Apply nest_asyncio
import nest_asyncio

nest_asyncio.apply()
load_dotenv(override=True)

local_llm = Ollama(model="llama3", request_timeout=600)
local_embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3",device="cuda")
Settings.llm = local_llm
Settings.embed_model = local_embed_model

#Setup the vector database
db_path = "./chroma_db"
db = chromadb.PersistentClient(path=db_path)
chroma_collection = db.get_or_create_collection("financial_reports")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store = vector_store)

#Extract Metadata based on the Dataset directory structure
#Metadata format - {"year": year, "company":company", source_filename":filename}
reports_dataset_path = "Financial_RAG_Chatbot"
dataset_metadata = []
companies = os.path.join(reports_dataset_path,"Financial Reports Dataset")
for company in companies:
    filenames = os.path.join(reports_dataset_path+"/"+company)
    for filename in filenames:
        year_match = re.search(r'(20\d{2})', filename)
        year = int(year_match.group(1)) if year_match else "Unknown"
        dataset = {"year":year,"company":company,"source_filename":filename}
        dataset_metadata.append(dataset)

documents = []
llama_parse_data_path = 'Financial_RAG_Chatbot/LlamaParsed_pdf_data'
filenames = os.listdir("Financial_RAG_Chatbot/LlamaParsed_pdf_data")
for filename, meta in zip(filenames,dataset_metadata):
    f = open(llama_parse_data_path + "/" + filename,'r',encoding="utf-8")
    full_text = f.read()
    doc = Document(text = full_text, metadata = meta)
    documents.append(doc)
    f.close()

#Context-Aware Chunking Step
#The parser is used to generate summaries of the table it finds in the documents.
#node_parser = MarkdownElementNodeParser(llm = local_llm,num_workers=1)
node_parser = MarkdownNodeParser()

#Chunking documents and extracting tables
nodes = node_parser.get_nodes_from_documents(documents)

#Embedding & Indexing
index = VectorStoreIndex(
            nodes = nodes,
            storage_context = storage_context
        )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)

@tool
def financial_data_retriever(query,filters):
    """
    Search financial documents.
    Args:
        query: The specific question (e.g. "Operating margin 2023").
        filters: (Optional) Any specific company or year mentioned.
    """
    full_query = f"{query}{filters}"
    response = query_engine.query(full_query)
    return str(response)

#Calculator tool
@tool
def calculator_tool(expression):
    """
    Evaluates the given mathematical expression.
    Args:
      expression: The mathematical expression given in string format to evaluate.
    """
    valid_chars = set("0123456789+-*/(). ")
    if not all(c in valid_chars for c in expression):
            return "Error: Invalid characters in math expression."
    return str(eval(expression))

ReadTimeout: timed out

In [ ]:
# DEFINE AGENT STATE
class AgentState(TypedDict):
    """
    Keeps track of the conversation flow.
    """
    user_query: str
    plan: List[str]
    insights: Annotated[List[str], operator.add] # Appends new insights to the list
    messages: List[Union[HumanMessage, AIMessage]]

llm = ChatOllama(model="llama3.1", temperature=0,timeout=300)

def planner_node(state: AgentState):
    """
    The Planner/Router: Breaks the user query into sub-tasks.
    """
    query = state["user_query"]

    # Prompt to force a list output
    prompt = f"""
    You are a Financial Lead. Break this query into 2-3 distinct steps for your analysts.
    Query: {query}

    Return ONLY a Python list of strings. Example: ["Retrieve 2023 revenue", "Retrieve 2022 revenue", "Calculate growth"]
    """
    response = llm.invoke(prompt)
    plan_text = response.content.strip("[]'\" ")
    plan = [s.strip().strip('"').strip("'") for s in plan_text.split(",")]

    return {"plan": plan, "messages": [AIMessage(content=f"Plan created: {plan}")]}

def analyst_node(state: AgentState):
    """
    The Worker: Executes the plan using tools.
    This node combines Quant/Qual/Sentiment roles for simplicity.
    """
    plan = state["plan"]
    insights = []

    # Bind tools to the LLM so it knows it can use them
    llm_with_tools = llm.bind_tools([financial_data_retriever, calculator_tool])

    for step in plan:
        # For each step, the LLM decides if it needs a tool
        msg = llm_with_tools.invoke(f"Execute this task: {step}. Use tools if needed.")

        if msg.tool_calls:
            for tool_call in msg.tool_calls:
                # Manually executing tool calls for transparency
                func_name = tool_call['name']
                args = tool_call['args']

                if func_name == 'financial_data_retriever':
                    result = financial_data_retriever.invoke(args)
                    insights.append(f"Data found for '{step}': {result}")
                elif func_name == 'calculator_tool':
                    result = calculator_tool.invoke(args)
                    insights.append(f"Calculation for '{step}': {result}")
        else:
            # If no tool needed, just use LLM's logic
            insights.append(msg.content)

    return {"insights": insights}

def synthesizer_node(state: AgentState):
    """
    The Writer: Compiles everything into a final report.
    """
    query = state["user_query"]
    insights = "\n".join(state["insights"])

    prompt = f"""
    You are a Senior Financial Analyst. Write a final response to the user query based ONLY on the gathered insights.

    User Query: {query}
    Gathered Insights:
    {insights}

    Format nicely with Markdown. If comparing numbers, use a table.
    """
    response = llm.invoke(prompt)
    return {"messages": [response]}

#  BUILD THE GRAPH (Orchestrator Logic)
workflow = StateGraph(AgentState)

workflow.add_node("planner", planner_node)
workflow.add_node("analyst", analyst_node)
workflow.add_node("synthesizer", synthesizer_node)

# Define the flow
workflow.set_entry_point("planner")
workflow.add_edge("planner", "analyst")
workflow.add_edge("analyst", "synthesizer")
workflow.add_edge("synthesizer", END)
app_graph = workflow.compile()

In [ ]:
from IPython.display import display, Markdown

def run_agentic_app(user_query):
    inputs = {
        "user_query": user_query,
        "plan": [],
        "insights": [],
        "messages": []
    }

    # Run the graph
    result = app_graph.invoke(inputs)

    # Extract final answer
    final_answer = result['messages'][-1].content

    print("\n" + "="*50)
    print("FINAL GENERATED REPORT")
    print("="*50 + "\n")

    # Render Markdown in Colab
    display(Markdown(final_answer))

# --- EXAMPLE USAGE ---
run_agentic_app("Compare the Operating Margins of TCS and Infosys for the latest available year.")